In [2]:
import sagemaker
from sagemaker.processing import ProcessingInput, ProcessingOutput, NetworkConfig, ScriptProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.s3 import S3Uploader

import sys
sys.path.append("/root/dme_sagemaker/dme_sagemaker")

from libs.config.config_vars import CONFIG, ENVIRONMENT

bucket = "us.com.syngenta.ap.nonprod"

In [3]:
sagemaker_session = sagemaker.Session(default_bucket=bucket)
role = CONFIG.get("role")

network_config = NetworkConfig(enable_network_isolation=False,
                               subnets = ["subnet-0327a8fda4133f6a9","subnet-0a9f652afd14455bb"],
                               security_group_ids = ["sg-00634d30526efcdec","sg-0405f346fdd704d24"])


processor = ScriptProcessor(command=['python3'],
    base_job_name="ap-nonprod-compute-bebid-advancement-decisions",
    sagemaker_session=sagemaker_session,
    role=role,
    instance_count=1,
    instance_type="ml.t3.large",
    image_uri=CONFIG.get('image_uri'),
    max_runtime_in_seconds=3600,
    network_config = network_config,
    output_kms_key = "arn:aws:kms:us-east-1:809800841141:key/353d6263-d454-444f-ac60-41afe025b445",
    tags = [{'Key': 'Application', 'Value': 'Advancement placement'},{'Key': 'Cost Center', 'Value': 'RDIT3000'}],
    env={'SAGEMAKER_ENV': ENVIRONMENT}
    )

inputs = [
    # Mount the src dir with your code here.
    ProcessingInput(
        input_name='libraries',
        source='/root/dme_sagemaker/dme_sagemaker/libs',
        destination="/opt/ml/processing/input/code/libs"
    )
]

outputs = [
    # Mount the src dir with your code here.
    ProcessingOutput(
        output_name='output_bebid_advancement_decisions',
        destination="s3://us.com.syngenta.ap.nonprod/poc/compute_bebid_advancement_decisions/data",
        source="/opt/ml/processing/data/bebid_advancement_decisions/"
    ),
]

In [ ]:
code_full_path = "/root/dme_sagemaker/dme_sagemaker/performance_pipeline/compute_init_script/src/processing.py"

processor.run(
    code=code_full_path,
    inputs=inputs,
    outputs=None,
    arguments=[
             '--ap_data_sector', 'CORN_NA_SUMMER',
             '--analysis_type','GenoPred',  
             '--analysis_year', '2021',
             '--target_pipeline_runid', '1341234',
             '--stages', '314',
            '--force_refresh','1'], # or a list, can't be an empty list.
    kms_key = "arn:aws:kms:us-east-1:809800841141:key/353d6263-d454-444f-ac60-41afe025b445",
)

INFO:sagemaker:Creating processing-job with name ap-nonprod-compute-bebid-advancement-de-2023-07-31-17-07-05-541


...................................................................

KeyboardInterrupt: 

In [ ]:
code_full_path = "/root/dme_sagemaker/dme_sagemaker/performance_pipeline/compute_rv_early_stg_selection_remark/src/processing.py"

processor.run(
    code=code_full_path,
    inputs=inputs,
    outputs=outputs,
    arguments=None, # or a list, can't be an empty list.
    kms_key = "arn:aws:kms:us-east-1:809800841141:key/353d6263-d454-444f-ac60-41afe025b445",
)
